# the vector space for the words will eventually change in the space sd they get close to the ones which has similar meaing 


In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#read the words
words = open('../names.txt', 'r').read().splitlines()

In [3]:
len(words)

32033

In [3]:
#get all non-repeating charcs in once
# build the vocal of chars and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi1 = {s:i+1 for i,s in enumerate(chars)}
stoi = {s:i for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [4]:
print(itos)

{0: '.', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


In [7]:
#re building the dataset
block_size = 3 #context_length; how many charactors do we take to predict the next one
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '----->' ,itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -----> e
[0, 0, 4]
..e -----> m
[0, 4, 12]
.em -----> m
[4, 12, 12]
emm -----> .
[12, 12, 0]
mm. -----> .
[12, 0, 0]
olivia
... -----> o
[0, 0, 14]
..o -----> l
[0, 14, 11]
.ol -----> i
[14, 11, 8]
oli -----> v
[11, 8, 21]
liv -----> i
[8, 21, 8]
ivi -----> .
[21, 8, 0]
vi. -----> .
[8, 0, 0]
ava
... -----> .
[0, 0, 0]
... -----> v
[0, 0, 21]
..v -----> .
[0, 21, 0]
.v. -----> .
[21, 0, 0]
isabella
... -----> i
[0, 0, 8]
..i -----> s
[0, 8, 18]
.is -----> .
[8, 18, 0]
is. -----> b
[18, 0, 1]
s.b -----> e
[0, 1, 4]
.be -----> l
[1, 4, 11]
bel -----> l
[4, 11, 11]
ell -----> .
[11, 11, 0]
ll. -----> .
[11, 0, 0]
sophia
... -----> s
[0, 0, 18]
..s -----> o
[0, 18, 14]
.so -----> p
[18, 14, 15]
sop -----> h
[14, 15, 7]
oph -----> i
[15, 7, 8]
phi -----> .
[7, 8, 0]
hi. -----> .
[8, 0, 0]


In [7]:
X.shape, X.dtype, Y.shape, Y.shape

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.Size([32]))

In [8]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  4],
        [ 0,  4, 12],
        [ 4, 12, 12],
        [12, 12,  0],
        [ 0,  0,  0],
        [ 0,  0, 14],
        [ 0, 14, 11],
        [14, 11,  8],
        [11,  8, 21],
        [ 8, 21,  8],
        [21,  8,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0, 21],
        [ 0, 21,  0],
        [ 0,  0,  0],
        [ 0,  0,  8],
        [ 0,  8, 18],
        [ 8, 18,  0],
        [18,  0,  1],
        [ 0,  1,  4],
        [ 1,  4, 11],
        [ 4, 11, 11],
        [11, 11,  0],
        [ 0,  0,  0],
        [ 0,  0, 18],
        [ 0, 18, 14],
        [18, 14, 15],
        [14, 15,  7],
        [15,  7,  8],
        [ 7,  8,  0]])

In [34]:
Y

tensor([ 4, 12, 12,  0,  0, 14, 11,  8, 21,  8,  0,  0,  0, 21,  0,  0,  8, 18,
         0,  1,  4, 11, 11,  0,  0, 18, 14, 15,  7,  8,  0,  0])

In [9]:
#lookup table
C = torch.randn((27,2)) #27 rows where each row is a represtative of each word and we embedd it to only 2 dim space
#the choioce of dimensiona;ity is hyperparameter

In [10]:
C

tensor([[-0.0601, -1.5389],
        [ 1.2137, -0.4272],
        [-0.0351,  0.0598],
        [-0.0528, -0.1820],
        [-0.7626, -0.7673],
        [ 1.1108,  1.5336],
        [ 1.2753,  0.7123],
        [-0.1926, -0.2704],
        [-1.0433,  0.0756],
        [-1.1127, -0.5630],
        [-0.1813, -0.0183],
        [ 0.6471, -0.7479],
        [ 0.0263, -0.4799],
        [ 0.0836, -0.8084],
        [-1.5254, -1.2260],
        [-0.5982, -0.7380],
        [-1.2120,  2.7358],
        [ 0.1547,  0.6749],
        [-0.0423, -0.6524],
        [-1.9166,  0.5864],
        [ 0.3511,  1.1407],
        [-1.8477,  0.3326],
        [-1.0796,  1.4184],
        [-0.3512, -0.6708],
        [-0.4367,  0.9630],
        [ 0.1533,  0.8782],
        [ 0.7475, -0.4097]])

In [22]:
F.one_hot(torch.tensor(5) ,num_classes=27).float() @ C #~ C[5]

tensor([1.1108, 1.5336])

In [16]:
X.dtype

torch.int64

In [23]:
emb = C[X]

In [33]:
emb.shape

torch.Size([32, 3, 2])

In [31]:
C.shape

torch.Size([27, 2])

In [32]:
X.shape

torch.Size([32, 3])

## build up the hidden layer

In [44]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [45]:
W1 = torch.randn(size=(6, 100)) # we have 3 input word of 2D according to the paper and what vector space we embed to here (c~2D) n 100 random neuron in the layer
b1 = torch.randn(100) 
W1.shape

torch.Size([6, 100])

In [51]:
#hidden Layer
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h.shape
# 32, 6
# 6, 100 

torch.Size([32, 100])

## final layer


In [54]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [56]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [57]:
counts = logits.exp()

In [62]:
prob = counts / counts.sum(1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [75]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(18.8451)

In [67]:
#-------------- CLEANER :) -------------------

In [68]:
X.shape, Y.shape #dataset

(torch.Size([32, 3]), torch.Size([32]))

In [71]:
g = torch.Generator().manual_seed(2147483647) 
C = torch.randn((27,2), generator= g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator = g)
parameters = [W1, W2, b1,b2, C]

In [72]:
sum(p.nelement() for p in parameters) #number of total parameters in total

3481

In [80]:
#forward pass
emb = C[X] #(32,3,2)
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) #32, 100
logits = h @ W2 + b2 # 32, 27
#counts = logits.exp()
#prob = counts / counts.sum(1, keepdim=True)
#loss = -prob[torch.arange(32), Y].log(.mean()
loss = F.cross_entropy(logits, Y)
#backward pass ;) 